In [1]:
# Importing the relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ensuring we see all the columns in the dataframe
pd.set_option('display.max_columns', 200)

In [2]:
# Importing the cleaned data
df = pd.read_parquet('../data/02_data_cleaning.parquet')

# Previewing the data
df.head()

,Event_Id,Investigation_Type,Accident_Number,Event_Date,Location,Country,Airport_Code,Airport_Name,Injury_Severity,Aircraft_damage,Aircraft_Category,Registration_Number,Make,Model,Amateur_Built,Number_of_Engines,Engine_Type,FAR_Description,Schedule,Purpose_of_flight,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_phase_of_flight,Report_Status,Publication_Date,Year,People_Onboard,Abbreviation,US_State
7,20020909X01562,Accident,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,None,BLACKBURN AG STRIP,Non-Fatal,Substantial,Airplane,N2482N,Cessna,140,No,1.0,Reciprocating,Part 91: General Aviation,None,Personal,0.0,0.0,0.0,2.0,VMC,Takeoff,Probable Cause,01-01-1982,1982,2,WA,Washington
8,20020909X01561,Accident,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,N58,HANOVER,Non-Fatal,Substantial,Airplane,N7967Q,Cessna,401B,No,2.0,Reciprocating,Part 91: General Aviation,None,Business,0.0,0.0,0.0,2.0,IMC,Landing,Probable Cause,01-01-1982,1982,2,NJ,New Jersey
9,20020909X01560,Accident,MIA82DA029,1982-01-01,"JACKSONVILLE, FL",United States,JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,None,N3906K,North American,NAVION L-17B,No,1.0,Reciprocating,None,None,Personal,0.0,0.0,3.0,0.0,IMC,Cruise,Probable Cause,01-01-1982,1982,3,FL,Florida
10,20020909X01559,Accident,FTW82DA034,1982-01-01,"HOBBS, NM",United States,None,None,Non-Fatal,Substantial,None,N44832,Piper,PA-28-161,No,1.0,Reciprocating,None,None,Personal,0.0,0.0,0.0,1.0,VMC,Approach,Probable Cause,01-01-1982,1982,1,NM,New Mexico
11,20020909X01558,Accident,ATL82DKJ10,1982-01-01,"TUSKEGEE, AL",United States,None,TUSKEGEE,Non-Fatal,Substantial,None,N4275S,Beech,V35B,No,1.0,Reciprocating,None,None,Personal,0.0,0.0,0.0,1.0,VMC,Landing,Probable Cause,01-01-1982,1982,1,AL,Alabama
